In [1]:
!pip install scrapy
!pip install Twisted==22.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 14.2 MB/s eta 0:00:00
  Obtaining dependency information for Twisted>=18.9.0 from https://files.pythonhosted.org/packages/2a/e3/9fe9cf016d32d050a2eec518c2f5156f7623b42e1ef3f2fa3e80c0ef654c/twisted-23.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for w3lib>=1.17.0 from https://files.pythonhosted.org/packages/82/e2/dcf8573d7153194eb673347cea1f9bbdb2a8e61030740fb6f50e4234a00b/w3lib-2.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for zope.interface>=5.1.0 from https://files.pythonhosted.org/packages/46/52/c881463f334126e12a5fb15d55e438a1a7e090f9e840c3c555bf3dca7091/zope.interface-6.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 8.8 MB/s eta 0:00:00
  Obtaining dependency information for protego>=0.1.15 from https://files.pythonhosted.org/packages/bc/16/14fd1ecdece2e1d87279fc09

In [5]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor
from bs4 import BeautifulSoup
from urllib.parse import urlparse, unquote

In [6]:
class FullPageSpider(scrapy.Spider):
    name = 'FullPageSpider'
    allowed_domains = ['lloydsbanking.com', 'lloydsbank.com']
    start_urls = ['https://www.lloydsbank.com/business/help-and-support/online-banking.html']
    custom_settings = {'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/527.3'}
    
    def parse(self, response):
        self.save_page(response)
        
        link_extractor = LinkExtractor(allow=r'/business/help-and-support/')
        for link in link_extractor.extract_links(response):
            yield scrapy.Request(link.url, callback=self.parse)
    
    def save_page(self, response):
        output_dir = 'output'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        parsed_url = urlparse(unquote(response.url))
        filename = os.path.basename(parsed_url.path)
        if filename.endswith('.html'):
            file_split = os.path.splitext(filename)
            filename = f'{file_split[0]}.txt'
        else:
            filename = f'{file_split}.txt'
        
        file_path = os.path.join(output_dir, filename)
        
        soup = BeautifulSoup(response.body, 'html.parser')
        
        nested_selector = 'body .main .content > div:nth-of-type(2) > div .container-fluid .row > div:nth-of-type(2)'
        with open(file_path, 'a', encoding='utf-8') as file:
            file.write(response.url + '\n')
            for div in soup.select(nested_selector):
                content = div.get_text(strip=False)
                file.write(content + '\n')
                
        if os.path.getsize('output/' + filename) == 0:
            nested_selector = 'body .main .content > div > div .container-fluid .row > div:nth-of-type(2)'
            with open(file_path, 'a', encoding='utf-8') as file:
                file.write(response.url + '\n')
                for div in soup.select(nested_selector):
                    content = div.get_text(strip=False)
                    file.write(content + '\n')

        if os.path.getsize('output/' + filename) == 0:
            nested_selector = 'body .main .content > div > div .container-fluid .row > div'
            with open(file_path, 'a', encoding='utf-8') as file:
                file.write(response.url + '\n')
                for div in soup.select(nested_selector):
                    content = div.get_text(strip=False)
                    file.write(content + '\n')

        self.log(f'saved file {file_path}')

process = CrawlerProcess()
process.crawl(FullPageSpider)
process.start()

2023-12-05 15:27:03 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-12-05 15:27:03 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) - [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.2.0 23 Nov 2023), cryptography 38.0.2, Platform Linux-4.19.0-25-cloud-amd64-x86_64-with-debian-10.13
2023-12-05 15:27:03 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit537.36 '
               '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/527.3'}
2023-12-05 15:27:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2023-12-05 15:27:03 [scrapy.extensions.telnet] INFO: Telnet Password: b08249be3be513bb
2023-12-05 15:27:03 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsol

ReactorNotRestartable: 